In [30]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
from pprint import pprint 

import torch

from collections import Counter

data = pd.read_csv('2015-to-2021.csv', encoding='utf8')
# data.head()
# data.columns
# effective_col = ('pitch_type release_speed release_pos_x release_pos_z batter pitcher events zone stand p_throws type hit_location bb_type ' +
# 'pfx_x pfx_z plate_x plate_z hit_distance_sc launch_speed launch_angle release_spin_rate').split()

c:\Users\DIAL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
data.columns

Index(['Unnamed: 0', 'pitch_type', 'game_date', 'release_speed',
       'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher',
       'events', 'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',


In [32]:
triplet = data['batter pitcher events hit_location bb_type'.split()]
triplet.hit_location = triplet.hit_location.fillna(0).astype(int) # NA as 0
triplet

c:\Users\DIAL\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,batter,pitcher,events,hit_location,bb_type
0,596019,519293,field_out,5,ground_ball
1,596019,519293,NaN,0,NaN
2,596019,519293,NaN,0,NaN
3,596019,519293,NaN,0,NaN
4,607043,519293,field_out,4,ground_ball
...,...,...,...,...,...
4695354,572761,452657,NaN,0,NaN
4695355,572761,452657,NaN,0,NaN
4695356,572761,452657,NaN,0,NaN
4695357,572761,452657,NaN,0,NaN


In [33]:
pprint(Counter(triplet.hit_location))
pprint(Counter(triplet.events))

Counter({0: 3635819,
         2: 278665,
         8: 142807,
         9: 126604,
         7: 125698,
         6: 107235,
         4: 96580,
         5: 87057,
         3: 58953,
         1: 35941})
Counter({nan: 3482864,
         'field_out': 481258,
         'strikeout': 267419,
         'single': 174331,
         'walk': 96497,
         'double': 54198,
         'home_run': 38585,
         'force_out': 24401,
         'grounded_into_double_play': 23417,
         'hit_by_pitch': 12347,
         'field_error': 8840,
         'sac_fly': 7694,
         'sac_bunt': 5757,
         'triple': 5335,
         'double_play': 2815,
         'fielders_choice': 2451,
         'fielders_choice_out': 2021,
         'intent_walk': 1923,
         'caught_stealing_2b': 1317,
         'strikeout_double_play': 939,
         'other_out': 249,
         'catcher_interf': 196,
         'sac_fly_double_play': 111,
         'caught_stealing_3b': 88,
         'pickoff_1b': 65,
         'caught_stealing_home': 4

In [34]:
new_triplet = triplet.assign(events_hit_location=triplet.events.map(str) + '_' + triplet.hit_location.map(str))

new_triplet['events_hit_location']

0          field_out_5
1                nan_0
2                nan_0
3                nan_0
4          field_out_4
              ...     
4695354          nan_0
4695355          nan_0
4695356          nan_0
4695357          nan_0
4695358          nan_0
Name: events_hit_location, Length: 4695359, dtype: object

In [35]:
# make label ((batter|pitcher)2vec)
from collections import Counter
event_cnt = Counter(new_triplet.events_hit_location)
# print(event_cnt)
events = sorted(event_cnt.items(), key=lambda x:x[1], reverse=True)
events_to_label = dict()
# print(events)
for event, n_event in events:
    if event.startswith('strikeout'):
        events_to_label[event] = 'K'
    elif event.startswith('walk'):
        events_to_label[event] = 'W'
    elif event.startswith('intent_walk'):
        events_to_label[event] = 'IW'
    elif event.startswith('balk'): # not reached
        events_to_label[event] = 'BK'
    elif event.startswith('hit_by_pitch'):
        events_to_label[event] = 'HP'
    
    elif event.startswith((
        'field_out', 'force_out', 'fielders_choice_out', 'other_out', 
        'double_play', 'grounded_into_double_play', 'triple_play',
        'sac_fly', 'sac_bunt', 'fielders_choice'
        )):
        events_to_label[event] = event[-1]
    elif event.startswith('field_error'):
        events_to_label[event] = 'E'+event[-1]
    elif event.startswith('single'):
        events_to_label[event] = 'S'+event[-1]
    elif event.startswith('double_0'):
        events_to_label[event] = 'DG'
    elif event.startswith('double'):
        events_to_label[event] = 'D'+event[-1]
    elif event.startswith('triple'):
        events_to_label[event] = 'T'+event[-1]
    elif event.startswith('home_run'):
        events_to_label[event] = 'HR'

    elif event.startswith('nan'):
        pass # no event occured
    elif event.startswith(('caught_stealing', 'pickoff', 'runner_double_play', 'stolen_base')):
        pass # not a batter event
    elif event.startswith(('passed_ball', 'wild_pitch', 'catcher_interf')):
        pass # not a batter event
    elif event.startswith(('ejection', 'game_advisory')):
        pass # not a batter event
    else:
        print('not classified', event, 'n_event =', event_cnt[event])

# from pprint import pprint
# pprint(event_cnt)
# pprint(events_to_label)

In [36]:
labels = sorted(list(set(events_to_label.values())))
print(labels)
print(events_to_label)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'D1', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'DG', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'HP', 'HR', 'IW', 'K', 'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'T0', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'W']
{'strikeout_2': 'K', 'walk_0': 'W', 'field_out_6': '6', 'field_out_4': '4', 'field_out_8': '8', 'field_out_5': '5', 'field_out_9': '9', 'field_out_7': '7', 'single_8': 'S8', 'field_out_3': '3', 'single_7': 'S7', 'single_9': 'S9', 'home_run_0': 'HR', 'field_out_1': '1', 'double_7': 'D7', 'double_9': 'D9', 'double_8': 'D8', 'hit_by_pitch_0': 'HP', 'force_out_6': '6', 'grounded_into_double_play_6': '6', 'single_6': 'S6', 'single_5': 'S5', 'field_out_2': '2', 'grounded_into_double_play_4': '4', 'force_out_5': '5', 'single_4': 'S4', 'force_out_4': '4', 'grounded_into_double_play_5': '5', 'single_1': 'S1', 'sac_fly_8': '8', 'double_0': 'DG', 'field_error_5': 'E5', 'sac_bunt_1': '1', 'triple_9': 'T9', '

In [37]:
# some weird events
data[(data.events == 'single') & (data.hit_location.isnull())]['game_date player_name batter pitcher events hit_location bb_type'.split()]

,game_date,player_name,batter,pitcher,events,hit_location,bb_type
291356,2021-07-21,"Kimbrel, Craig",425877,518886,single,NaN,fly_ball
613639,2021-04-26,"Rogers, Trevor",649966,669432,single,NaN,ground_ball
652726,2021-04-16,"Pressly, Ryan",664034,519151,single,NaN,line_drive
2530754,2017-10-09,"Sale, Chris",594828,519242,single,NaN,ground_ball
2876154,2017-07-06,"Gaviglio, Sam",622194,543208,single,NaN,fly_ball
4656245,2015-04-16,"Young, Chris",573627,432934,single,NaN,ground_ball


In [38]:
import torch
from pybaseball import playerid_reverse_lookup

def get_player_info(player_data, limit=1000):
    player_vc = player_data.value_counts()
    probable_players = player_vc[player_vc > limit].index
    df = playerid_reverse_lookup(probable_players)
    name_last, name_first = df['name_last'], df['name_first']
    df['full_name'] = name_first.str.capitalize() + ' ' + name_last.str.capitalize()
    return df[['key_mlbam', 'full_name']]

def get_player_name(player_db, key): # 타자 및 투수 합해야?
    name_df = player_db[player_db.key_mlbam == key].full_name
    if name_df.empty:
        # raise Exception(f'no player name found for {key}')
        return None
    else:
        return name_df.values[0]

limit = 100 # 1000구 이상 대결한 선수들

prob_batters = get_player_info(data['batter'], limit)
prob_pitchers = get_player_info(data['pitcher'], limit)

display(prob_batters)
display(prob_pitchers)

print(get_player_name(prob_batters, 660271))
print(get_player_name(prob_pitchers, 660271))

n_batter = len(prob_batters)
n_pitcher = len(prob_pitchers)
n_dim = 16
print(n_batter, n_pitcher, n_dim)

,key_mlbam,full_name
0,547989,José Abreu
1,554429,Dustin Ackley
2,660670,Ronald Acuna
3,542436,Cristhian Adames
4,642715,Willy Adames
...,...,...
1385,605548,Bradley Zimmer
1386,475582,Ryan Zimmerman
1387,519455,Jordan Zimmermann
1388,450314,Ben Zobrist


,key_mlbam,full_name
0,430911,David Aardsma
1,472551,Fernando Abad
2,676265,Cory Abbott
3,656061,Albert Abreu
4,650556,Bryan Abreu
...,...,...
1564,669145,Bruce Zimmermann
1565,519455,Jordan Zimmermann
1566,217096,Barry Zito
1567,676604,Tyler Zuber


Shohei Ohtani
Shohei Ohtani
1390 1569 16


In [39]:
'''
player_dict = {
    'batter': torch.nn.Embedding(n_batter, 1) # with fixed integer value (which is unique and sorted by ascending order?)
    'pitcher : torch.nn.Embedding(n_pitcher, 1) # with fixed integer value (which is unique and sorted by ascending order?)
}

'''

import torch.nn as nn

# class PlayerIdx(nn.Module):
#     def __init__(self, player_ids):
#         super().__init__()
#         self.player_ids = torch.from_numpy(player_ids.to_numpy()).long()

#     def forward(self, x):
#         return (self.player_ids == x).nonzero()

class Batter_Pitcher_2_Vec(nn.Module):
    def __init__(self, n_batter, n_pitcher, n_dim, n_result, sparse=True):
        super().__init__()
        # self.player_dict = player_dict # 'batter': tensor of player id with shape (n_batter, ), 'pitcher': tensor of player id with shape (n_pitcher, )
        self.batter_emb = nn.Embedding(n_batter, n_dim, sparse=sparse)
        self.pitcher_emb = nn.Embedding(n_pitcher, n_dim, sparse=sparse)
        self.linear = nn.Linear(2*n_dim, n_result)
        
    def forward(self, x):
        # batter, pitcher, events triplet training
        # batter_idx = self.player_dict['batter'](x[:,0])
        # pitcher_idx = self.player_dict['pitcher'](x[:,1])
        batter_idx, pitcher_idx = x[:,0], x[:,1]
        result = x[:,2]

        batter_emb = self.batter_emb(batter_idx)
        pitcher_emb = self.pitcher_emb(pitcher_idx)

        merge_emb = torch.cat([batter_emb, pitcher_emb], dim=1)
        y_pred = self.linear(merge_emb)
        print('y_pred.shape', y_pred.shape)

        return torch.softmax(y_pred, dim=1)


In [40]:
# batter_idx = PlayerIdx(prob_batters.key_mlbam)
# print(batter_idx(torch.tensor([[660670], [605548]]).long()))

import torch.nn as nn
from tqdm import tqdm

# display(prob_batters)
# display(prob_pitchers)

# a = [[val, i] for i, val in enumerate(prob_batters.key_mlbam.to_list())]
# print(a)
n_batter = prob_batters.key_mlbam.max()
n_pitcher = prob_pitchers.key_mlbam.max()
n_dim = 32
n_result = len(labels)

batter_np = prob_batters.key_mlbam.to_numpy()
pitcher_np = prob_pitchers.key_mlbam.to_numpy()

x_input = []
# display(new_triplet)
print(prob_batters.key_mlbam.to_numpy())
print(prob_pitchers.key_mlbam.to_numpy())

okay = set()
not_okay = set()
for line in tqdm(new_triplet[['batter', 'pitcher', 'events_hit_location']].to_numpy()):
    # print(line)
    if line[0] in batter_np and line[1] in pitcher_np:
        if line[-1] in events_to_label:
            # print([line[0], line[1], events_to_label[line[-1]]])
            x_input.append([line[0], line[1], labels.index(events_to_label[line[-1]])])
            okay.add(line[-1])

print(okay)
print(not_okay)

[547989 554429 660670 ... 519455 450314 572287]
[430911 472551 676265 ... 217096 676604 543964]


100%|██████████| 4695359/4695359 [00:32<00:00, 144120.46it/s]

{'field_error_4', 'double_play_5', 'sac_fly_9', 'triple_play_6', 'double_play_2', 'force_out_1', 'triple_9', 'double_play_3', 'triple_play_4', 'field_error_2', 'other_out_8', 'sac_fly_double_play_7', 'fielders_choice_9', 'force_out_6', 'force_out_4', 'double_play_7', 'fielders_choice_0', 'single_3', 'single_8', 'sac_fly_8', 'grounded_into_double_play_2', 'strikeout_0', 'field_error_3', 'triple_play_1', 'double_0', 'fielders_choice_4', 'triple_8', 'double_play_9', 'force_out_8', 'double_6', 'triple_6', 'sac_bunt_3', 'sac_fly_double_play_9', 'home_run_4', 'field_error_8', 'sac_bunt_5', 'sac_bunt_1', 'triple_3', 'home_run_9', 'single_4', 'fielders_choice_5', 'walk_0', 'triple_7', 'sac_fly_5', 'double_3', 'grounded_into_double_play_5', 'walk_1', 'sac_bunt_4', 'fielders_choice_out_1', 'double_play_4', 'sac_bunt_double_play_1', 'home_run_7', 'walk_5', 'field_error_9', 'force_out_2', 'single_5', 'sac_bunt_6', 'grounded_into_double_play_4', 'sac_bunt_double_play_2', 'fielders_choice_8', 'field

In [41]:
'''
x = [[605548, 676265, 1], [572287, 217096, 4]]

x = torch.tensor(x).long().cuda()
val = model.forward(x)
'''

from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

data_loader = DataLoader(x_input, batch_size=512, shuffle=True)

epochs = 100
lr = 0.001

model = Batter_Pitcher_2_Vec(n_batter, n_pitcher, n_dim, n_result)
model.cuda()

optimizer = Adam(model.parameters(), lr=lr)
criterion = CrossEntropyLoss()

for epoch in range(epochs):
    loss = 0
    for x_batch in data_loader:
        x_batch = x_batch.cuda()
        model_output = model(x_batch)
        print(model_output.shape, x_batch[:,2].shape)
        batch_loss = criterion(model_output, x_batch[:,2])
        batch_loss.backward()
        loss += batch_loss.item()
    print(f'epoch: {epoch}, loss: {loss}')



RuntimeError: CUDA error: device-side assert triggered